In [1]:
import pandas as pd
import json
import os

In [2]:
with open('../.foursquare_API.json', 'r') as file:
    secrets = json.load(file)

secrets

{'token': 'fsq3D/VHyHU4HVfbaRITsOl0pIswAP2pr//Ac/m+gD5H1TI=',
 'client_id': 'Q1B4DZHK1Q4YYMMQXYH1J4N3CKDH5DLR5FGNTHILBZD0BXVO',
 'client_secret': 'AJWWWAR2WHOQDLLESW4DHRQTXUFEPBOILFHR5XJSATEQZR5D'}

1. Read dataset

In [3]:
dirName = '/Users/mikhailgasanov/Downloads/EVdatasets/PaloAlto/'
fnames = os.listdir(dirName)
df = pd.read_csv(os.path.join(dirName, fnames[0]))

In [8]:
coords = df[['Latitude', 'Longitude']].drop_duplicates()

In [13]:
CLIENT_ID = secrets['client_id']
CLIENT_SECRET = secrets['client_secret']

LIMIT = 100

RADIUS = 1000

Latitude = coords.iloc[0, :]['Latitude']

Longitude = coords.iloc[0, :]['Longitude']
VERSION = '20180604'
url = f"""
https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={Latitude},{Longitude}&v={VERSION}&radius={RADIUS}&limit={LIMIT}
"""

In [14]:
import pandas as pd # for data processing
import folium   #for creating maps
import requests  #for retreiving Information from URL
from geopy.geocoders import Nominatim  #converting address to cordinates
from pandas.io.json import json_normalize #converting json to DataFrame 

In [15]:
# reading radius from user
# radius = input("Enter the radius for searching : ")
# reading search limit from user
# Limit = input("enter the Limit for Results to display : ")
# url  = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, Limit)
results = requests.get(url).json()

In [16]:
results

{'meta': {'code': 410,
  'errorType': 'deprecated',
  'errorDetail': 'Usage of the V2 Places API has been deprecated for new Projects. Please see our updated documentation for V3 for more details: https://docs.foursquare.com/reference',
  'requestId': '6304fc3893d61027eb9f61c8'},
 'response': {}}

In [ ]:
items = results['response']['groups'][0]['items']
items[0]

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby['venue.categories'] = nearby.apply(get_category_type, axis=1)

# clean columns
nearby.columns = [col.split('.')[-1] for col in nearby.columns]

#replce NaN values with Not found in address
nearby['address'] = nearby['address'].fillna("Not found")
nearby.head(10)      

In [ ]:
# creating map zoom at present Location
map = folium.Map(location=[latitude,longitude],zoom_start=14)

# adding the current location to map
folium.CircleMarker(location=[latitude,longitude],radius=5,color='red',fill=True,fill_color='red').add_to(map)
folium.Marker(location=[latitude,longitude],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map)

# drawing radius of search
folium.Circle(location=[latitude,longitude],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map)

# adding all the popular places around
for lat,lon,name,cate,addr,dist in zip(nearby.lat,nearby.lng,nearby.name,nearby.categories,nearby.address,nearby.distance):
    folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map)
    folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map)

map